<a href="https://colab.research.google.com/github/baharaaii/DBCourseInha/blob/main/group_d_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sqlite3
import pandas as pd

# Connect to SQLite database (creates a new database if it doesn't exist)
conn = sqlite3.connect('group_d_assignment.db')
cursor = conn.cursor()

# Create Employees table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Employees (
        EmployeeID INTEGER PRIMARY KEY,
        Name TEXT,
        Position TEXT,
        DepartmentID INTEGER,
        Salary REAL
    )
''')

# Create Departments table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Departments (
        DepartmentID INTEGER PRIMARY KEY,
        DepartmentName TEXT
    )
''')

# Create Projects table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Projects (
        ProjectID INTEGER PRIMARY KEY,
        ProjectName TEXT,
        DepartmentID INTEGER
    )
''')

# Create Assignments table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Assignments (
        EmployeeID INTEGER,
        ProjectID INTEGER,
        HoursWorked INTEGER,
        PRIMARY KEY (EmployeeID, ProjectID),
        FOREIGN KEY (EmployeeID) REFERENCES Employees (EmployeeID),
        FOREIGN KEY (ProjectID) REFERENCES Projects (ProjectID)
    )
''')

# Commit the changes and close the connection
conn.commit()


In [ ]:
import sqlite3

# Connect to the SQLite database
conn = sqlite3.connect('group_d_assignment.db')

# Task 1: List all employees along with the names of the projects they are working on.
query_task1 = """
    SELECT
        E.EmployeeID,
        E.Name AS EmployeeName,
        A.ProjectID,
        P.ProjectName
    FROM
        Employees E
    JOIN
        Assignments A ON E.EmployeeID = A.EmployeeID
    JOIN
        Projects P ON A.ProjectID = P.ProjectID;
"""

# Then we execute the query and fetch the results
cursor = conn.execute(query_task1)
result_task1 = cursor.fetchall()

# Print the results
print("Task 1: List of employees and their projects")
for row in result_task1:
    print(row)

# Task 2: Create a subquery to find the department with the highest average salary.
query_task2 = """
    SELECT
        D.DepartmentID,
        D.DepartmentName,
        AVG(E.Salary) AS AvgDepartmentSalary
    FROM
        Employees E
    JOIN
        Departments D ON E.DepartmentID = D.DepartmentID
    GROUP BY
        D.DepartmentID, D.DepartmentName
    HAVING
        AVG(E.Salary) = (SELECT MAX(AvgSalary) FROM (SELECT AVG(Salary) AS AvgSalary FROM Employees GROUP BY DepartmentID));
"""

# Then we execute the query and fetch the results
cursor = conn.execute(query_task2)
result_task2 = cursor.fetchall()

# Print the results
print("\nTask 2: Department with the highest average salary")
for row in result_task2:
    print(row)

# Commit the changes and close the connection
conn.commit()

Task 1: List of employees and their projects

Task 2: Department with the highest average salary


In [ ]:
# A window function ranking employees in each department based on their salary

# Definition of a SQL query to perform our task.
# A command SELECT is used for choosing what to print. In that case there're EmployeeID, Name, DepartmentID, Salary
# A command RANK () OVER is used to rank by salary in descending order and they divided by department with a command PARTITION BY
win_func = """
  SELECT EmployeeID, Name, DepartmentID, Salary,
  RANK() OVER (PARTITION BY DepartmentID ORDER BY Salary DESC) AS DepartmentRank
  FROM Employees;
  """
  # Create a variable to store result of performed query and print it's being read to pandas DataFrame
win_func_df = pd.read_sql_query(win_func, conn)
win_func_df

,EmployeeID,Name,DepartmentID,Salary,DepartmentRank


In [30]:
conn = sqlite3.connect('group_d_assignment.db')
cursor = conn.cursor()

# Creating a trigger to update total hours when a new assignment is added
trigger_sql = '''
CREATE TRIGGER IF NOT EXISTS UpdateTotalHoursAfterAssignment
AFTER INSERT ON Assignments
FOR EACH ROW
BEGIN
    -- Update 'TotalHours' in 'Projects' table
    UPDATE Projects
    SET TotalHours = TotalHours + NEW.HoursWorked
    WHERE ProjectID = NEW.ProjectID;
END;
'''

try:
    # Execute the SQL code
    cursor.executescript(trigger_sql)

    # Commit the changes
    conn.commit()
    print("Trigger created successfully.")
except sqlite3.Error as e:
    # Handle any errors that may occur during execution
    print("Error:", e)
finally:
    # Close the connection
    conn.close()


Trigger created successfully.
